In [3]:
# %load '~/playground/basic.py'
import os
from pathlib import Path
from IPython.display import display as ipd

import numpy as np
import scipy
import pandas as pd
import pandas_profiling

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sklearn

from datetime import datetime as dt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.figure(figsize=(20,10))
mpl.rcParams['figure.figsize'] = (20, 5)
import mypackage
import funniest

%load_ext autoreload
%autoreload 2

sns.set_context("poster")
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style("whitegrid")
cm = sns.light_palette("green", as_cmap=True)

# exec(open("./shared.py").read())
# import shared

In [4]:
print(funniest.joke())

Wenn ist das Nunstück git und Slotermeyer? doch! ... ab das Oder die Flipperwaldt gersput.


In [5]:
from mypackage import skeleton
skeleton.fib(5)

5

# Load Dataset

In [6]:
import pandas as pd
from sklearn.datasets import load_boston
boston = load_boston()

data = pd.DataFrame(boston.data,columns=boston.feature_names)
data['target'] = pd.Series(boston.target)
ipd(data.sample(5))

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data.iloc[:,-1], test_size=10, random_state=42)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
88,0.05660,0.0,3.41,0.0,0.489,7.007,86.3,3.4217,2.0,270.0,17.8,396.90,5.50,23.6
356,8.98296,0.0,18.10,1.0,0.770,6.212,97.4,2.1222,24.0,666.0,20.2,377.73,17.60,17.8
458,7.75223,0.0,18.10,0.0,0.713,6.301,83.7,2.7831,24.0,666.0,20.2,272.21,16.23,14.9
66,0.04379,80.0,3.37,0.0,0.398,5.787,31.1,6.6115,4.0,337.0,16.1,396.90,10.24,19.4
223,0.61470,0.0,6.20,0.0,0.507,6.618,80.8,3.2721,8.0,307.0,17.4,396.90,7.60,30.1


/Users/nkreiling/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Normal Training

In [25]:
from sklearn.linear_model import LinearRegression
regr_one_feature = LinearRegression()
regr_one_feature.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [26]:
from sklearn.metrics import mean_squared_error # Same as the computation above the plot
mean_squared_error(y_test, regr_one_feature.predict(x_test))

9.203879048168957

In [27]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, regr_one_feature.predict(x_test))

0.8743356986050591

In [28]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, regr_one_feature.predict(x_test))

2.4988473085061393

# Integrate ModelDB

In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
import pkg_resources
pkg_resources.require("modeldb==0.0.1a30")
import modeldb.sklearn_native.ModelDbSyncer as mdb
from modeldb.sklearn_native import SyncableMetrics

from sklearn.datasets import load_boston
boston = load_boston()

data = pd.DataFrame(boston.data,columns=boston.feature_names)
data['target'] = pd.Series(boston.target)

name = "ModelDB Brownbag"
author = "Nico"
description = "using Bosten Housing Dataset"
syncer_obj = mdb.Syncer(
    mdb.NewOrExistingProject(name, author, description),
    mdb.DefaultExperiment(),
    mdb.NewExperimentRun("Abc"))

# test_data, train_data = mdb.cross_validation.train_test_split_sync(data, test_size=0.8, random_state=42)
x_train, x_test, y_train, y_test = mdb.cross_validation.train_test_split_sync(data.iloc[:,:-1], data.iloc[:,-1], test_size=10, random_state=42)

In [10]:
import modeldb.sklearn_native.ModelDbSyncer as mdb
linreg = mdb.linear_model.LinearRegression()
linreg.fit_sync(x_train, y_train)
test_pred = linreg.predict_sync(x_test)

import modeldb.sklearn_native.SyncableMetrics
from sklearn.metrics import mean_absolute_error
accuracy = SyncableMetrics.compute_metrics(linreg, mean_absolute_error, y_test, test_pred, data.iloc[:,:-1].values,"predictionCol", 'target')
syncer_obj.sync()

In [11]:
model = mdb.linear_model.Ridge()
model.fit_sync(x_train, y_train)
test_pred = model.predict_sync(x_test)

accuracy = SyncableMetrics.compute_metrics(model, mean_absolute_error, y_test, test_pred, data.iloc[:,:-1].values,"predictionCol", 'target')
syncer_obj.sync()

In [12]:
model = mdb.linear_model.ElasticNet()
model.fit_sync(x_train, y_train)
test_pred = model.predict_sync(x_test)

accuracy = SyncableMetrics.compute_metrics(model, mean_absolute_error, y_test, test_pred, data.iloc[:,:-1].values,"predictionCol", 'target')
syncer_obj.sync()

In [16]:
lasso = mdb.linear_model.Lasso()
lasso.fit_sync(x_train,y_train)
test_pred = lasso.predict_sync(x_test)

accuracy = SyncableMetrics.compute_metrics(lasso, mean_absolute_error, y_test, test_pred, data.iloc[:,:-1].values,"predictionCol", 'target')
syncer_obj.sync()

# Cross-Validation

In [14]:
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor()
scores = cross_val_score(clf, data.iloc[:,:-1], data.iloc[:,-1], cv=5)
ipd(scores)
print("MAE: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) 
accuracy = SyncableMetrics.compute_metrics(clf, mean_absolute_error, y_test, test_pred, data.iloc[:,:-1].values,"predictionCol", 'target')

array([0.7339495 , 0.86318965, 0.7040027 , 0.49518488, 0.16083559])

MAE: 0.59 (+/- 0.49)


# Hyper Parameter Tuning

In [6]:
from sklearn.grid_search import GridSearchCV
from sklearn import datasets
from sklearn.metrics import precision_score
from sklearn.svm import SVC

from modeldb.sklearn_native.ModelDbSyncer import *
from modeldb.sklearn_native import SyncableMetrics

# This is a sample usage of GridSearch in scikit, adapted from
# http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
name = "grid search"
author = "srinidhi"
description = "digits dataset"
syncer_obj = Syncer(
    NewOrExistingProject(name, author, description),
    DefaultExperiment(),
    NewExperimentRun("Abc"))

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

# Split the dataset in two equal parts
x_train, x_test, y_train, y_test = cross_validation.train_test_split_sync(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5)
clf.fit_sync(x_train, y_train)

print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
y_pred = clf.predict_sync(x_test)
mean_error = SyncableMetrics.compute_metrics(
    clf, precision_score, y_test, y_pred, x_test, '', '')

syncer_obj.sync()

The model is trained on the full development set.
The scores are computed on the full evaluation set.


/Users/nkreiling/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


## Regression

In [3]:
import sklearn
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

data = pd.DataFrame(boston.data,columns=datasets.load_boston().feature_names)
data['target'] = pd.Series(datasets.load_boston().target)

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data.iloc[:,-1], test_size=10, random_state=42)

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()

tuned_parameters = {
    'n_jobs': (1,2,3)
}

model = sklearn.linear_model.LinearRegression()

clf = GridSearchCV(model, tuned_parameters)
clf.fit(x_train,y_train)
predictions = clf.predict(x_test)

print(sorted(clf.cv_results_.keys()))
from sklearn.metrics import mean_squared_error # Same as the computation above the plot
mean_squared_error(y_test, predictions)

['mean_fit_time', 'mean_score_time', 'mean_test_score', 'mean_train_score', 'param_n_jobs', 'params', 'rank_test_score', 'split0_test_score', 'split0_train_score', 'split1_test_score', 'split1_train_score', 'split2_test_score', 'split2_train_score', 'std_fit_time', 'std_score_time', 'std_test_score', 'std_train_score']


9.2038790481688579

### With ModelDB

In [9]:
# pkg_resources.require("modeldb==0.0.1a31")
import sklearn
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import pkg_resources
import modeldb.sklearn_native.ModelDbSyncer as mdb

import pandas as pd

from sklearn.datasets import load_boston
boston = load_boston()
data = pd.DataFrame(boston.data,columns=datasets.load_boston().feature_names)
data['target'] = pd.Series(datasets.load_boston().target)


from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data.iloc[:,-1], test_size=10, random_state=42)


from modeldb.sklearn_native.ModelDbSyncer import *
from modeldb.sklearn_native import SyncableMetrics

name = "ModelDB Brownbag - Regression 2"
author = "srinidhi"
description = "digits dataset"
syncer_obj = mdb.Syncer(
    mdb.NewOrExistingProject(name, author, description),
    mdb.DefaultExperiment(),
    mdb.NewExperimentRun("test"))


tuned_parameters = {
    'n_jobs': (1,2,3)
}

model = mdb.linear_model.LinearRegression()

clf = mdb.GridSearchCV(model, tuned_parameters)
clf.fit_sync(x_train,y_train)
predictions = clf.predict_sync(x_test)
print(predictions)
# print(sorted(clf.cv_results_.keys()))
from sklearn.metrics import mean_squared_error # Same as the computation above the plot
print(mean_squared_error(y_test, predictions))

syncer_obj.sync()

[ 29.20350997  36.24928191  13.86750133  24.63174372  18.61348409
  23.0655571   17.6437391   13.57232198  23.13303812  20.54041793]
9.20387904817


In [18]:
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

import modeldb.sklearn_native.ModelDbSyncer as mdb

from modeldb.sklearn_native.ModelDbSyncer import *
from modeldb.sklearn_native import SyncableMetrics

# Uses GridSearch and Pipeline objects in scikit, adapted from
# http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html
name = "ModelDB Brownbag - Regression"
author = "srinidhi"
description = "digits dataset"
syncer_obj = mdb.Syncer(
    mdb.NewOrExistingProject(name, author, description),
    mdb.DefaultExperiment(),
    mdb.NewExperimentRun("Abcde"))

digits = mdb.datasets.load_digits()
x = digits.data[:1000]
y = digits.target[:1000]

# parameters = {
#     'alpha': (0.00001, 0.000001)
# }

# tuned_parameters = [{'n_jobs': [1]},
#                     {'n_jobs': [2,3]}]

tuned_parameters = {
    'n_jobs': (1,2,3)
}

model = sklearn.linear_model.LinearRegression()

# from mdb.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error # Same as the computation above the plot
clfGrid = GridSearchCV(model, tuned_parameters, cv=5, scoring=mean_squared_error)
clfGrid.fit(x_train, y_train)
# clfGrid.fit_sync(data.iloc[:,:-1], data.iloc[:,-1])

syncer_obj.sync()

dist =  {
    'alpha': scipy.stats.expon(scale=100)
}
# clfRandom = RandomizedSearchCV(
#     model, dist, cv=None, scoring='%s_weighted' % 'precision')

# clfRandom.fit_sync(x, y)
# syncer_obj.sync()

/Users/nkreiling/miniconda3/lib/python3.6/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/nkreiling/miniconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
/Users/nkreiling/miniconda3/lib/python3.6/site-packages/sklearn/metrics/metrics.py:4: DeprecationWarning: sklearn.metrics.metrics is deprecated and will be removed in 0.18. Please import from sklearn.metrics
  DeprecationWarning)


ImportError: cannot import name 'bincount'

## Classification

In [20]:
import modeldb.sklearn_native.SyncableMetrics
from sklearn.metrics import mean_absolute_error

In [21]:
from sklearn.datasets import load_iris
iris = pd.DataFrame(load_iris().data,columns=load_iris().feature_names)
iris['target'] = pd.Series(load_iris().target)
ipd(iris.sample(5))

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
105,7.6,3.0,6.6,2.1,2
145,6.7,3.0,5.2,2.3,2
43,5.0,3.5,1.6,0.6,0
51,6.4,3.2,4.5,1.5,1
12,4.8,3.0,1.4,0.1,0


In [22]:
import modeldb.sklearn_native.ModelDbSyncer as mdb
from modeldb.sklearn_native import SyncableMetrics

name = "ModelDB Brownbag - Classification"
author = "Nico"
description = "using iris Dataset"
class_syncer = mdb.Syncer(
    mdb.NewOrExistingProject(name, author, description),
    mdb.DefaultExperiment(),
    mdb.NewExperimentRun("Abcd"))

# test_data, train_data = mdb.cross_validation.train_test_split_sync(data, test_size=0.8, random_state=42)
x_train, x_test, y_train, y_test = mdb.cross_validation.train_test_split_sync(iris.iloc[:,:-1], iris.iloc[:,-1], test_size=10, random_state=42)

model = mdb.linear_model.LogisticRegression()
model.fit_sync(x_train, y_train)
test_pred = model.predict_sync(x_test)

accuracy = SyncableMetrics.compute_metrics(model, mean_absolute_error, y_test, test_pred, data.iloc[:,:-1].values,"predictionCol", 'target')
class_syncer.sync()

In [23]:
model = sklearn.svm.SVC()
model = modeldb.sklearn_native.ModelDbSyncer.SVC()
model.fit_sync(x_train, y_train)
test_pred = model.predict_sync(x_test)

accuracy = SyncableMetrics.compute_metrics(model, mean_absolute_error, y_test, test_pred, data.iloc[:,:-1].values,"predictionCol", 'target')
syncer_obj.sync()

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.pipeline import Pipeline

import modeldb.sklearn_native.ModelDbSyncer as mdb

from modeldb.sklearn_native.ModelDbSyncer import *
from modeldb.sklearn_native import SyncableMetrics

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

# clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5)
# clf.fit_sync(x_train, y_train)
model = SVC()
clfGrid = GridSearchCV(model, tuned_parameters, cv=5)

clfGrid.fit_sync(data.iloc[:,:-1], data.iloc[:,-1])
y_pred = clfGrid.predict_sync(x_test)

from sklearn.metrics import precision_score
mean_error = SyncableMetrics.compute_metrics(
    clfGrid, precision_score, y_test, y_pred, x_test, '', '')

class_syncer.sync()

dist =  {
    'alpha': scipy.stats.expon(scale=100)
}
# clfRandom = RandomizedSearchCV(
#     model, dist, cv=None, scoring='%s_weighted' % 'precision')

# clfRandom.fit_sync(x, y)
# class_syncer_obj.sync()

AttributeError: 'GridSearchCV' object has no attribute 'fit_sync'

In [ ]:
# Model Selection
dist =  {'alpha': scipy.stats.expon(scale=100)}
gs = skl.grid_search.RandomizedSearchCV(lasso, dist)
gs.fit_sync(x_train,y_train)
syncer_obj.sync()

test_pred = gs.predict(x_test)

from sklearn.metrics import mean_absolute_error
accuracy = SyncableMetrics.compute_metrics(gs, mean_absolute_error, y_test, test_pred, data.iloc[:,:-1].values,"predictionCol", 'target')

In [ ]:
lasso = mdb.linear_model.Lasso()

In [ ]:

from modeldb.basic.Structs import (
    Model, ModelConfig, ModelMetrics, Dataset)
from modeldb.basic.ModelDbSyncerBase import Syncer

# Create a syncer using a convenience API
syncer_obj = Syncer.create_syncer("Sample Project", \
                                  "test_user", \
                                  "use sync_all", \
                                  host="backend")


datasets = {
    "train": Dataset("/path/to/train", {"num_cols": 15, "dist": "random"}),
    "test": Dataset("/path/to/test", {"num_cols": 15, "dist": "gaussian"})
}
model = "model_obj"
mdb_model1 = Model("NN", model, "/path/to/model1")
model_config1 = ModelConfig("NN", {"l1": 10})
model_metrics1 = ModelMetrics({"accuracy": 0.8})

mdb_model2 = Model("NN", model, "/path/to/model2")
model_config2 = ModelConfig("NN", {"l1": 20})
model_metrics2 = ModelMetrics({"accuracy": 0.9})

syncer_obj.sync_datasets(datasets)

syncer_obj.sync_model("train", model_config1, mdb_model1)
syncer_obj.sync_metrics("test", mdb_model1, model_metrics1)

syncer_obj.sync_model("train", model_config2, mdb_model2)
syncer_obj.sync_metrics("test", mdb_model2, model_metrics2)

syncer_obj.sync()

# Other

## List all SKLearn estimator with method fit and predict

In [15]:
from sklearn.utils.testing import all_estimators
i = 0
for name, class_ in all_estimators():
    if hasattr(class_, 'fit') and hasattr(class_, 'predict'):
        i+=1
        print(class_)
i

<class 'sklearn.linear_model.bayes.ARDRegression'>
<class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>
<class 'sklearn.ensemble.weight_boosting.AdaBoostRegressor'>
<class 'sklearn.cluster.affinity_propagation_.AffinityPropagation'>
<class 'sklearn.ensemble.bagging.BaggingClassifier'>
<class 'sklearn.ensemble.bagging.BaggingRegressor'>
<class 'sklearn.linear_model.bayes.BayesianRidge'>
<class 'sklearn.naive_bayes.BernoulliNB'>
<class 'sklearn.cluster.birch.Birch'>
<class 'sklearn.cross_decomposition.cca_.CCA'>
<class 'sklearn.calibration.CalibratedClassifierCV'>
<class 'sklearn.mixture.dpgmm.DPGMM'>
<class 'sklearn.tree.tree.DecisionTreeClassifier'>
<class 'sklearn.tree.tree.DecisionTreeRegressor'>
<class 'sklearn.linear_model.coordinate_descent.ElasticNet'>
<class 'sklearn.linear_model.coordinate_descent.ElasticNetCV'>
<class 'sklearn.tree.tree.ExtraTreeClassifier'>
<class 'sklearn.tree.tree.ExtraTreeRegressor'>
<class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
<class 's

83

In [6]:
from sklearn.grid_search import GridSearchCV
from sklearn import datasets
from sklearn.metrics import precision_score
from sklearn.svm import SVC

from modeldb.sklearn_native.ModelDbSyncer import *
from modeldb.sklearn_native import SyncableMetrics

# This is a sample usage of GridSearch in scikit, adapted from
# http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
name = "grid search"
author = "srinidhi"
description = "digits dataset"
syncer_obj = Syncer(
    NewOrExistingProject(name, author, description),
    DefaultExperiment(),
    NewExperimentRun("Abc"))

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

# Split the dataset in two equal parts
x_train, x_test, y_train, y_test = cross_validation.train_test_split_sync(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(SVC(), tuned_parameters, cv=5)
clf.fit_sync(x_train, y_train)

print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
y_pred = clf.predict_sync(x_test)
mean_error = SyncableMetrics.compute_metrics(
    clf, precision_score, y_test, y_pred, x_test, '', '')

syncer_obj.sync()

ModuleNotFoundError: No module named 'modeldb.sklearn_native'

In [ ]:
import os
import sys
os.getcwd() 
print(sys.executable)
sys.path